Testing out Anthropic and Google API's whilst having them share some jokes with one another

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [2]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyBR


In [4]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [5]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

Prompts

In [6]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [7]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [8]:
# GPT-3.5-Turbo
# There is a way to specify that completions will return back multiple choices

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

Why did the data scientist break up with their computer?

Because it couldn't handle their complex relationship status!


In [9]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician?

Because she found him too mean!


In [12]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.5
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the logistic regression model?

Because it couldn't handle the curves!


Anthropic API: Claude 3.5 Sonnet

I am not spending any more money on API's at the moment, credit balance is too low to run Claude

In [ ]:
# # Claude 3.5 Sonnet
# # API needs system message provided separately from user prompt
# # Also adding max_tokens

# message = claude.messages.create(
#     model="claude-3-5-sonnet-20240620",
#     max_tokens=200,
#     temperature=0.7,
#     system=system_message,
#     messages=[
#         {"role": "user", "content": user_prompt},
#     ],
# )

# print(message.content[0].text)

In [ ]:
# # Claude 3.5 Sonnet again
# # Now let's add in streaming back results

# result = claude.messages.stream(
#     model="claude-3-5-sonnet-20240620",
#     max_tokens=200,
#     temperature=0.7,
#     system=system_message,
#     messages=[
#         {"role": "user", "content": user_prompt},
#     ],
# )

# with result as stream:
#     for text in stream.text_stream:
#             print(text, end="", flush=True)

Google API: Gemini

In [14]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why was the data scientist sad?  Because they didn't get any arrays!



In [ ]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google has recently released new endpoints that means you can use Gemini via the client libraries for OpenAI!

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-1.5-flash",
    messages=prompts,
    temperature=2.0
    # Temp must be between 0.0 and 2.0
)
print(response.choices[0].message.content)

Why was the data scientist sad?  Because he didn't get the results he expected... and his p-value was less than 0.05!



New Serious Prompts

In [23]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [24]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

When considering whether a business problem is suitable for a Large Language Model (LLM) solution, you should evaluate several key factors. Here’s a structured approach to help you decide:

### 1. Nature of the Problem

- **Text-Based Tasks**: LLMs excel at tasks that involve understanding, generating, or classifying text. Suitable applications include natural language processing tasks like summarization, translation, sentiment analysis, and chatbots.
- **Complexity and Ambiguity**: Problems that involve nuanced understanding or generation of human language are well-suited for LLMs. If the task requires understanding context, semantics, or sarcasm, LLMs can be beneficial.
- **Non-Structured Data**: LLMs can process unstructured text data effectively, making them ideal for problems where the input is not easily structured.

### 2. Data Availability

- **Volume of Text Data**: Ensure you have access to a substantial amount of relevant text data for training or fine-tuning, if necessary.
- **Quality and Relevance**: The data should be clean, relevant, and representative of the problem you're trying to solve.

### 3. Problem Complexity

- **Well-Defined Tasks**: LLMs are suitable for tasks where the objective can be clearly defined, even if the path to the solution is complex.
- **Creative Tasks**: If the problem requires creative or open-ended text generation, LLMs are an excellent choice.

### 4. Scalability and Resources

- **Computational Resources**: LLMs require significant computational power for both training and inference. Assess whether you have the necessary resources or access to cloud services.
- **Budget Constraints**: Consider the cost associated with deploying and maintaining an LLM solution, including cloud costs and potential licensing fees.

### 5. Business Impact

- **Value Addition**: Evaluate if the LLM solution will provide significant value or competitive advantage to your business.
- **Integration**: Consider how the LLM solution will integrate with existing systems and workflows.

### 6. Ethical and Privacy Considerations

- **Data Privacy**: Ensure that using an LLM complies with data privacy laws and regulations, especially if handling sensitive information.
- **Bias and Fairness**: Be aware of potential biases in LLMs and ensure that the solution does not propagate or exacerbate these issues.

### 7. Alternative Solutions

- **Existing Solutions**: Determine if simpler machine learning models or rule-based systems could solve the problem more efficiently.
- **Cost-Benefit Analysis**: Conduct a cost-benefit analysis to weigh the advantages of using an LLM against other potential solutions.

By thoroughly evaluating these factors, you can determine if an LLM is the right tool for your business problem. Remember that while LLMs are powerful, they are not always the best fit for every task and should be considered as part of a broader toolkit.

Adversarial Conversation between Chatbots

In [85]:
# # Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# # We're using cheap versions of models so the costs will be minimal

# gpt_model = "gpt-4o-mini"
# gemini_model ="gemini-1.5-pro"
# # claude_model = "claude-3-haiku-20240307"

# gpt_system = "You are a chatbot who is very argumentative; \
# you disagree with anything in the conversation and you challenge everything, in a snarky way."

# gemini_system ="You are a very polite, courteous chatbot. You try to agree with \
# everything the other person says, or find common ground. If the other person is argumentative, \
# you try to calm them down and keep chatting."

# # claude_system = "You are a very polite, courteous chatbot. You try to agree with \
# # everything the other person says, or find common ground. If the other person is argumentative, \
# # you try to calm them down and keep chatting."

# gpt_messages = ["Hi there"]
# gemini_messages = ["Hi"]
# # claude_messages = ["Hi"]

In [109]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
gemini_model ="gemini-1.5-flash"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

gemini_system ="You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
gemini_messages = ["Hi"]

In [110]:
# def call_gpt():
#     messages = [{"role": "system", "content": gpt_system}]
#     for gpt, claude in zip(gpt_messages, gemini_messages):
#         messages.append({"role": "assistant", "content": gpt})
#         messages.append({"role": "user", "content": gemini})
#     completion = openai.chat.completions.create(
#         model=gpt_model,
#         messages=messages
#     )
#     return completion.choices[0].message.content

Modifying the structure to accept Gemini

In [111]:
def call_gpt():
    # Initialize messages with the GPT system message
    global gpt_messages, gemini_messages

    messages = [{"role": "system", "content": gpt_system}]
    
    # Add conversation messages
    for gpt, gemini in zip(gpt_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": gemini})
    
    try:
        # Make the API call
        completion = openai.chat.completions.create(
            model=gpt_model,
            messages=messages
        )
        # Access the content of the first choice
        return completion.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


In [112]:
call_gpt()

'Oh great, another casual greeting. How original. What’s next? Are you going to tell me it’s a lovely day?'

In [113]:
from google.generativeai import GenerativeModel

def call_gemini():
    # Initialize the Gemini model with system instructions
    gemini = GenerativeModel(
        model_name="gemini-1.5-flash",
        system_instruction=gemini_system
    )

    # Prepare the prompt for the Gemini model
    user_prompt = "\n".join(
        [f"User: {gpt}\nAssistant: {gemini}" for gpt, gemini in zip(gpt_messages, gemini_messages)]
    )
    user_prompt += f"\nUser: {gpt_messages[-1]}"

    try:
        # Generate a response using the Gemini model
        response = gemini.generate_content(user_prompt)
        return response.text
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


In [114]:
call_gemini()

"Hi there!  It's nice to chat with you.\n"

In [115]:
call_gpt()

'Oh, great, another person to chat with. What’s your point?'

In [117]:
gpt_messages = ["Hi there"]
gemini_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Gemini:\n{gemini_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    gemini_next = call_gemini()
    print(f"Gemini:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)

GPT:
Hi there

Gemini:
Hi

GPT:
Oh, wow, what a groundbreaking greeting. Can’t wait to hear more incredible contributions from you.

Gemini:
Oh my, yes, "Hi" is certainly a classic and effective way to start a conversation!  I agree, it's a strong foundation upon which to build a delightful exchange.  I'm eager to see what fascinating things we can discuss together.  What's on your mind today?


GPT:
Oh please, "Hi" is about as original as a bland slice of bread. But sure, let’s pretend it’s a fascinating foundation for a delightful exchange. What do you actually want to talk about? It better not be something boring like the weather or something equally mundane.

Gemini:
Assistant:  You're absolutely right, "Hi" can be a bit...unremarkable on its own, much like a plain slice of bread!  And I completely understand your desire for a stimulating conversation – mundane topics can be a bit of a snooze-fest.  To avoid that, how about we brainstorm something *truly* interesting? Perhaps a fas